In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from scipy import stats
import datetime as dt
from datetime import timedelta
from sklearn.tree import DecisionTreeRegressor
from matplotlib.animation import FuncAnimation
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.formula.api as smf
import scipy.stats as stats


In [6]:
SPP_path = r'C:\Users\felip\Desktop\Electricity\Energy Market\Energy Market (SPP)'

def add_info(df):
    intervals = df[df.columns[0]].values.tolist()
    dates = []
    times = []
    weekday = []
    months = []
    days = []
    hour = []
    for interval in intervals:
        date = interval.split(' ')[0]
        try:
            date = dt.datetime.strptime(date,'%Y-%m-%d').date()
        except:
            date = dt.datetime.strptime(date,'%m/%d/%Y').date()            
        dates.append(date)
        months.append(date.month)
        days.append(date.day)
        if date.weekday() < 5:
            weekday.append(True)
        else:
            weekday.append(False)
        time = interval.split(' ')[1].split('.')[0]
        time = dt.datetime.strptime(time,'%H:%M:%S').time()
        times.append(time)
        hour.append(dt.time(time.hour))
    df['Local Date'] = np.array(dates)
    df['Local Time'] = np.array(times)
    df['Hour'] = np.array(hour)
    df['Weekday'] = np.array(weekday)
    df['Month'] = np.array(months)
    df['Day'] = np.array(days)
    return df

def GMT2CT(s):
    date = s.split('T')[0]
    date = dt.datetime.strptime(date,'%Y-%m-%d').date()
    time = s.split('T')[1][:-1]
    hour = int(time.split(':')[0])
    if hour >= 6:
        hour = hour - 6
    else:
        hour = 24 + (hour - 6)
        date = date - timedelta(1)
    time = str(hour) + ':' + time.split(':')[1] + ':' + time.split(':')[2]
    time = dt.datetime.strptime(time,'%H:%M:%S').time()
    return [date, time]

In [9]:
gen_mix_2018 = pd.read_csv(SPP_path + '\Generation Mix By Fuel Type\GenMix_2018.csv')
list_intervals = gen_mix_2018[gen_mix_2018.columns[0]].values.tolist()
local_time = []
local_date = []
for value in list_intervals:
    local_date.append(GMT2CT(value)[0])
    local_time.append(GMT2CT(value)[1])
gen_mix_2018['Local Date'] = np.array(local_date)
gen_mix_2018['Local Time'] = np.array(local_time)

In [10]:
days_31 = []
for n in range(1,10):
    days_31.append('0'+str(n))
for n in range(10,32):
    days_31.append(str(n))
cal_dict = {'01':days_31,
            '02':days_31[0:28],
            '03':days_31,
           '04':days_31[0:-1],
            '05':days_31,
            '06':days_31[0:-1],
           '07':days_31,
            '08':days_31,
            '09':days_31[0:-1],
           '10':days_31,
           '11':days_31[0:-1],
           '12':days_31}

In [11]:
# Opening and concatenating RT datasets
path = r'C:\Users\felip\Desktop\Electricity\Energy Market\Energy Market (SPP)\RT\2018'
RT_path = 'RTBM-LMP-DAILY-SL-2018'
end = '.csv'
dfs = []
for key in cal_dict.keys():
    for value in cal_dict[key]:
        dfs.append(pd.read_csv(path+'\\'+key+'\\By_Day\\'+RT_path+key+value+end))
RT = pd.concat(dfs)
RT.head()

,Interval,GMT Interval,Settlement Location Name,PNODE Name,LMP,MLC,MCC,MEC
0,01/01/2018 00:05:00,01/01/2018 06:05:00,AEC,SOUC,255.8769,5.4654,0.0000,250.4115
1,01/01/2018 00:05:00,01/01/2018 06:05:00,AECC_CSWS,CSWS_AECC_LA,262.5802,5.0312,7.1375,250.4115
2,01/01/2018 00:05:00,01/01/2018 06:05:00,AECC_ELKINS,CSWSELKINSUNELKINS_RA,270.2234,3.0020,16.8099,250.4115
3,01/01/2018 00:05:00,01/01/2018 06:05:00,AECC_FITZHUGH,CSWSFITZHUGHPLT1,257.2724,6.8609,0.0000,250.4115
4,01/01/2018 00:05:00,01/01/2018 06:05:00,AECC_FLTCREEK,CSWSFLINTCRKUN1_JOU_AECC_RA,266.5657,-3.0577,19.2119,250.4115


In [12]:
RT2018_aggloc = RT.groupby('Interval')[['Interval',' LMP']].agg({'Interval':'first',
                                                                    ' LMP':'mean'})
RT2018 = add_info(RT2018_aggloc)
RT2018 = RT2018.rename(columns={' LMP':'LMP'})
RT2018.head()

,Interval,LMP,Local Date,Local Time,Hour,Weekday,Month,Day
Interval,,,,,,,,
01/01/2018 00:05:00,01/01/2018 00:05:00,246.567610,2018-01-01,00:05:00,00:00:00,True,1,1
01/01/2018 00:10:00,01/01/2018 00:10:00,64.864068,2018-01-01,00:10:00,00:00:00,True,1,1
01/01/2018 00:15:00,01/01/2018 00:15:00,164.686048,2018-01-01,00:15:00,00:00:00,True,1,1
01/01/2018 00:20:00,01/01/2018 00:20:00,173.867912,2018-01-01,00:20:00,00:00:00,True,1,1
01/01/2018 00:25:00,01/01/2018 00:25:00,163.353003,2018-01-01,00:25:00,00:00:00,True,1,1


In [13]:
# Opening and concatenating DA datasets
path = r'C:\Users\felip\Desktop\Electricity\Energy Market\Energy Market (SPP)\DA\2018'
DA_path = 'DA-LMP-SL-2018'
end = '0100.csv'
dfs = []
for key in cal_dict.keys():
    for value in cal_dict[key]:
        dfs.append(pd.read_csv(path+'\\'+key+'\\By_Day\\'+DA_path+key+value+end))
DA2018 = pd.concat(dfs)
DA2018.head()

,Interval,GMTIntervalEnd,Settlement Location,Pnode,LMP,MLC,MCC,MEC
0,01/01/2018 01:00:00,01/01/2018 07:00:00,AEC,SOUC,39.6809,0.8665,0.5970,38.2174
1,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_CSWS,CSWS_AECC_LA,38.3723,0.7331,-0.5781,38.2173
2,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_ELKINS,CSWSELKINSUNELKINS_RA,38.4189,0.4093,-0.2077,38.2173
3,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_FITZHUGH,CSWSFITZHUGHPLT1,38.5076,1.0540,-0.7637,38.2173
4,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_FLTCREEK,CSWSFLINTCRKUN1_JOU_AECC_RA,37.8395,-0.4387,0.0609,38.2173


In [14]:
DA2018_aggloc = DA2018.groupby('Interval')[['Interval','LMP']].agg({'Interval':'first',
                                                                    'LMP':'mean'})
DA2018_aggloc = add_info(DA2018_aggloc)
DA2018.head()

,Interval,GMTIntervalEnd,Settlement Location,Pnode,LMP,MLC,MCC,MEC
0,01/01/2018 01:00:00,01/01/2018 07:00:00,AEC,SOUC,39.6809,0.8665,0.5970,38.2174
1,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_CSWS,CSWS_AECC_LA,38.3723,0.7331,-0.5781,38.2173
2,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_ELKINS,CSWSELKINSUNELKINS_RA,38.4189,0.4093,-0.2077,38.2173
3,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_FITZHUGH,CSWSFITZHUGHPLT1,38.5076,1.0540,-0.7637,38.2173
4,01/01/2018 01:00:00,01/01/2018 07:00:00,AECC_FLTCREEK,CSWSFLINTCRKUN1_JOU_AECC_RA,37.8395,-0.4387,0.0609,38.2173


In [15]:
def ssr(prediction, test):
    return ((prediction - test)**2).sum()
def pe(prediction, test):
    return (abs((prediction - test)/test))*100
def filler(df, DA):
    price_list = df[DA].values.tolist()
    value = 0.0
    new_list = []
    for price in price_list:
        if math.isnan(price)==True:
            new_list.append(value)
        else:
            value = price
            new_list.append(price)
    df[DA] = np.array(new_list)
    return df

In [16]:
comparison = DA2018_aggloc.iloc[:,0:2].join(RT2018,how='right',lsuffix='_DA',rsuffix='_RT')
DART2018_5min = filler(comparison, 'LMP_DA')
gen_2018 = gen_mix_2018
new_index = []
for i in range(gen_2018.shape[0]):
    new_index.append(gen_2018['Local Date'].iloc[i].strftime("%m/%d/%Y") + ' ' + gen_2018['Local Time'].iloc[i].strftime("%H:%M:%S"))
gen_2018.index = np.array(new_index)
DART_gen_2018 = gen_2018.join(DART2018_5min,how='right',lsuffix='_gen',rsuffix='_price')
DART_gen_2018 = DART_gen_2018.rename(columns={' Average Actual Load':'Load',
                                             ' Wind Self':'Wind',
                                             ' Coal Market':'Coal_Mkt',
                                             ' Coal Self':'Coal_Self',
                                             'Local Time_price':'Local Time'})

In [35]:
def ssr_split4(df, s):
    if df.shape[0]>(288/4):
        smallest_y = 0
        smallest_x = 0
        for i in range(0, df.shape[0]):
            try:
                y_avg1 = df[s][0:i+1].values.mean()
                y_avg2 = df[s][i:df.shape[0]].values.mean()
                ssr = ((df[s][0:i+1].values - y_avg1)**2).sum() + ((df[s][i:df.shape[0]].values - y_avg2)**2).sum()
                if i == 0:
                    smallest_y = ssr
                if ssr < smallest_y:
                    smallest_y = ssr
                    smallest_x = i
            except:
                y_avg1 = df[s].values.mean()
                ssr = ((df[s][0:i+1].values - y_avg1)**2).sum()
                if ssr < smallest_y:
                    smallest_y = ssr
                    smallest_x = i
        smallest = df.iloc[0:smallest_x+1]
        largest = df.iloc[smallest_x+1:]
        if smallest.shape[0] > largest.shape[0]:
            temp = smallest
            smallest = largest
            largest = temp
        concat = [smallest] + [ssr_split4(largest, s)]
        return concat
    else:
        return df

def unnest(l, empty_l):
    for i in l:
        if type(i) == list:
            unnest(i, empty_l)
        else:
            empty_l.append(i)
            
def unnested_df(l):
    for df in l:
        means = np.empty(df.shape[0])
        means.fill(df['LMP_RT'].mean())
        df['Means'] = means
    unnested = pd.concat(l)
    unnested['Time'] = unnested.index
    unnested = unnested.sort_values('Time')
    return unnested

def sklearn_clusters(train_df, test_df):
    # converting datetime to float
    minutes = []
    for time in train_df.index:
        minutes.append(time.hour * 60 + time.minute + 0.1)
    train_df['minutes'] = np.array(minutes)
    minutes = []
    for time in test_df.index:
        minutes.append(time.hour * 60 + time.minute + 0.1)
    test_df['minutes'] = np.array(minutes)
    # testing existing module
    X = train_df['minutes'].values.reshape(-1,1)
    y = train_df['LMP_RT']
    # Fit regression model
    regr_1 = DecisionTreeRegressor(max_depth=3)
    regr_1.fit(X, y)
    # Predict
    X_test = test_df['minutes'].values.reshape(-1,1)
    y_1 = regr_1.predict(X_test)
    test_df['sklearn cluster'] = y_1
    price_leaves = []
    for price in y_1:
        if price not in price_leaves:
            price_leaves.append(price)
    branch_dfs = []
    for price in price_leaves:
        branch_dfs.append(test_df[test_df['sklearn cluster']==price])
    return branch_dfs

In [36]:
# system-wide, 5 predictors
# multi or single clustered -- comparings R^2

n = 10
custom_PE_array = np.empty(n)
custom_R2_array = np.empty(n)
sklearn_PE_array = np.empty(n)
sklearn_R2_array = np.empty(n)

check = 0

for i in range(n):
    custom_PE_list = []
    custom_R2_list = []
    sklearn_PE_list = []
    sklearn_R2_list = []

    for j in range(1,13):
        by_month = DART_gen_2018[DART_gen_2018['Month']==j]

        train, test = train_test_split(by_month, test_size=0.33)

        train_data = train.groupby('Local Time')[['LMP_RT','LMP_DA','Load','Wind']].mean()
        train_data['RT_sem'] = train.groupby('Local Time')['LMP_RT'].sem()
        train_data['DA_sem'] = train.groupby('Local Time')['LMP_DA'].sem()        

        test_data = test.groupby('Local Time')[['LMP_RT','LMP_DA','Load','Wind']].mean()
        test_data['RT_sem'] = test.groupby('Local Time')['LMP_RT'].sem()
        test_data['DA_sem'] = test.groupby('Local Time')['LMP_DA'].sem() 

        test_data_copy = test_data.copy()
        '''
        size = test_data_copy.index.shape[0]
        test_data_copy['fitted RT (custom)'] = np.zeros(size)            
        cl = []
        unnest(ssr_split4(train_data, 'LMP_RT'), cl)
        for cluster in cl:
            result = smf.ols(formula="LMP_RT ~ LMP_DA + RT_sem + DA_sem + Load + Wind", data=cluster).fit()
            custom_R2_list.append(result.rsquared)
            for hour in cluster.index:
                test_data_copy['fitted RT (custom)'].loc[hour] = ( 
                                            test_data_copy['Wind'].loc[hour]*result.params[5] + 
                                            test_data_copy['Load'].loc[hour]*result.params[4] + 
                                            test_data_copy['DA_sem'].loc[hour]*result.params[3] + 
                                            test_data_copy['RT_sem'].loc[hour]*result.params[2] + 
                                            test_data_copy['LMP_DA'].loc[hour]*result.params[1] + 
                                            result.params[0])
        custom_PE_list.append(pe(test_data_copy['fitted RT (custom)'], test_data_copy['LMP_RT']).mean())
        '''
                
        branch_dfs = sklearn_clusters(train_data, test_data)
        size = test_data_copy.index.shape[0]
        test_data_copy['fitted RT (sklearn)'] = np.zeros(size) 
        for cluster in branch_dfs:
            result = smf.ols(formula="LMP_RT ~ LMP_DA + RT_sem + DA_sem + Load + Wind", data=cluster).fit()
            sklearn_R2_list.append(result.rsquared)
            for hour in cluster.index:
                test_data_copy['fitted RT (sklearn)'].loc[hour] = ( 
                                            test_data_copy['Wind'].loc[hour]*result.params[5] + 
                                            test_data_copy['Load'].loc[hour]*result.params[4] + 
                                            test_data_copy['DA_sem'].loc[hour]*result.params[3] + 
                                            test_data_copy['RT_sem'].loc[hour]*result.params[2] + 
                                            test_data_copy['LMP_DA'].loc[hour]*result.params[1] + 
                                            result.params[0])
        sklearn_PE_list.append(pe(test_data_copy['fitted RT (sklearn)'], test_data_copy['LMP_RT']).mean())
            
            
        #check += np.where(test_data_copy['fitted RT (custom)']==0.0,1,0).sum()
        check += np.where(test_data_copy['fitted RT (sklearn)']==0.0,1,0).sum()
        if check > 0:
            print(check)
            break
                
    print('{}%'.format(10*i))

    custom_PE_array[i] = np.array(custom_PE_list).mean()
    custom_R2_array[i] = np.array(custom_R2_list).mean()
    sklearn_PE_array[i] = np.array(sklearn_PE_list).mean()
    sklearn_R2_array[i] = np.array(sklearn_R2_list).mean()


print('custom cluster:\n\tMAPE: {};\n\tMean R^2: {};'.format(custom_PE_array.mean(),custom_R2_array.mean()))
print('sklearn clusters:\n\tMAPE: {};\n\tMean R^2: {};'.format(sklearn_PE_array.mean(),sklearn_R2_array.mean()))
print(check)

C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


0%


C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarni

10%


C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


...

C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


80%


C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1543: RuntimeWarn

90%
custom cluster:
	MAPE: nan;
	Mean R^2: nan;
sklearn clusters:
	MAPE: 7.546185164076293;
	Mean R^2: nan;
0


C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice.
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\felip\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: RuntimeWarning: Mean of empty slice.


In [29]:
sklearn_PE_array

array([ 7.24700444,  7.80138442,  7.11314897,  7.31673611,  7.14428853,
        6.91159795,  7.39024725,  8.51540832,  8.70948002, 13.13738251])

In [30]:
custom_PE_array

array([14.24192378, 16.18190899, 14.7118502 , 13.59620738, 13.09013312,
       13.35126824, 15.58325798, 14.3093914 , 26.1351283 , 89.03435118])

In [38]:
result.pvalues

Intercept    0.739192
LMP_DA       0.430566
RT_sem       0.040589
DA_sem       0.138412
Load         0.492253
Wind         0.284085
dtype: float64